# Lab 9: Human in the Loop - Aprovação Humana para Ações Críticas

<img src="./assets/LC_HITL.png" width="300">

## 🎯 O que você vai aprender neste Lab

**Human in the Loop (HITL)** permite que humanos intervenham no fluxo do agente, aprovando ou rejeitando ações antes que sejam executadas.

### Por que HITL é importante?

1. **Segurança**: Prevenir ações destrutivas ou irreversíveis
2. **Controle**: Manter humanos no comando de decisões críticas
3. **Auditoria**: Criar trail de aprovações para compliance
4. **Confiança**: Usuários confiam mais em agentes com supervisão

### Quando usar HITL?

| Situação | Exemplo |
|----------|---------|
| Operações destrutivas | DELETE, DROP TABLE |
| Ações financeiras | Transferências, compras |
| Comunicação externa | Enviar emails, notificações |
| Mudanças de configuração | Alterar permissões |

### Fluxo HITL:

```
┌─────────────────────────────────────────────────────────────┐
│                    FLUXO HITL                                │
│                                                              │
│   Usuário pergunta                                          │
│        ↓                                                     │
│   Agente decide chamar tool                                 │
│        ↓                                                     │
│   ⚠️ INTERRUPT: "Executar execute_sql?"                     │
│        ↓                                                     │
│   Humano decide:                                            │
│        ├── APPROVE → Tool executa → Continua                │
│        └── REJECT  → Mensagem de erro → Para                │
└─────────────────────────────────────────────────────────────┘
```

### Decisões possíveis:

| Decisão | Efeito |
|---------|--------|
| `approve` | Executa a tool normalmente |
| `reject` | Cancela com mensagem de erro |
| `edit` | Permite modificar os argumentos (avançado) |

## ⚙️ Setup - Configuração do Ambiente

In [ ]:
%pip install -U mlflow>=3 langchain>=1 langchain-community langgraph databricks-langchain --quiet
dbutils.library.restartPython()

In [ ]:
import mlflow
mlflow.langchain.autolog()

In [ ]:
# No Databricks, usamos spark.sql() diretamente
# Verifique as tabelas disponíveis no catálogo NASA GCN
%sql
SHOW TABLES IN nasa_gcn.silver

## 🔧 Configuração da Tool e System Prompt

In [ ]:
# RuntimeContext não é necessário quando usamos spark diretamente

In [ ]:
from langchain_core.tools import tool


@tool
def execute_sql(query: str) -> str:
    """Execute a SQL command on Databricks and return results."""
    try:
        result = spark.sql(query).limit(5).toPandas()
        return result.to_markdown()
    except Exception as e:
        return f"Error: {e}"

In [ ]:
SYSTEM_PROMPT = """Você é um analista cuidadoso de Databricks SQL especializado em dados da NASA GCN.

Regras:
- Pense passo a passo.
- Quando precisar de dados, chame a ferramenta `execute_sql` com UMA query SELECT.
- Apenas leitura; sem INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limite a 5 linhas de saída, a menos que o usuário peça mais.
- Se a ferramenta retornar 'Error:', revise o SQL e tente novamente.
- Prefira listas explícitas de colunas; evite SELECT *.
- Use o catálogo nasa_gcn e schema silver (nasa_gcn.silver.tabela).
- Se o banco de dados estiver offline, peça ao usuário para tentar novamente mais tarde.
"""

## 🛡️ Configurando o Middleware HITL

O `HumanInTheLoopMiddleware` define:
- **interrupt_on**: Quais tools requerem aprovação
- **allowed_decisions**: Que decisões são permitidas (`approve`, `reject`, `edit`)

### Componentes necessários:
1. **Checkpointer**: Para salvar estado enquanto aguarda aprovação
2. **HumanInTheLoopMiddleware**: O middleware de interrupção
3. **thread_id**: Para retomar a conversa após decisão

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from databricks_langchain import ChatDatabricks

# Inicializar o modelo Databricks
llm = ChatDatabricks(
    endpoint="databricks-meta-llama-3-3-70b-instruct",
    temperature=0.1,
)

agent = create_agent(
    model=llm,
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={"execute_sql": {"allowed_decisions": ["approve", "reject"]}},
        ),
    ],
)

## ❌ Exemplo 1: REJEIÇÃO

Quando rejeitamos uma ação, passamos uma mensagem explicando o motivo. O agente recebe essa mensagem e adapta sua resposta.

### Fluxo:
1. Pergunta dispara tool call
2. `__interrupt__` aparece no resultado
3. Enviamos `Command(resume={"decisions": [{"type": "reject", "message": "motivo"}]})`
4. Agente responde com base na rejeição

In [ ]:
from langgraph.types import Command

question = "Liste todos os eventos do catálogo nasa_gcn schema silver"

config = {"configurable": {"thread_id": "1"}}

result = agent.invoke(
    {"messages": [{"role": "user", "content": question}]},
    config=config,
)

if "__interrupt__" in result:
    description = result['__interrupt__'][-1].value['action_requests'][-1]['description']
    print(f"\033[1;3;31m{80 * '-'}\033[0m")
    print(
        f"\033[1;3;31m Interrupt:{description}\033[0m"
    )

    # Simulando REJEIÇÃO - database offline
    result = agent.invoke(
        Command(
            resume={
                "decisions": [{"type": "reject", "message": "o banco de dados está offline."}]
            }
        ),
        config=config,  # Same thread ID to resume the paused conversation
    )
    print(f"\033[1;3;31m{80 * '-'}\033[0m")

print(result["messages"][-1].content)

## ✅ Exemplo 2: APROVAÇÃO

Quando aprovamos, a tool é executada normalmente. O loop `while` continua até não haver mais interrupts.

### Fluxo:
1. Pergunta dispara tool call
2. `__interrupt__` aparece
3. Aprovamos com `{"type": "approve"}`
4. Tool executa, pode haver mais interrupts
5. Repete até resposta final

In [ ]:
# Agora com APROVAÇÃO automática
question = "Quais são os tipos de eventos mais frequentes no nasa_gcn.silver?"

config = {"configurable": {"thread_id": "2"}}

result = agent.invoke(
    {"messages": [{"role": "user", "content": question}]},
    config=config,
)

while "__interrupt__" in result:
    description = result['__interrupt__'][-1].value['action_requests'][-1]['description']
    print(f"\033[1;3;31m{80 * '-'}\033[0m")
    print(
        f"\033[1;3;31m Interrupt:{description}\033[0m"
    )
    
    # Aprovando a execução
    result = agent.invoke(
        Command(
            resume={"decisions": [{"type": "approve"}]}
        ),
        config=config,  # Same thread ID to resume the paused conversation
    )

for msg in result["messages"]:
    msg.pretty_print()

## 📚 Resumo e Conclusão do Curso

### O que aprendemos neste Lab:

| Conceito | Descrição |
|----------|-----------|
| **HumanInTheLoopMiddleware** | Middleware que pausa para aprovação |
| **interrupt_on** | Define quais tools requerem aprovação |
| **Command(resume=...)** | Retoma execução com decisão |
| **approve/reject** | Decisões disponíveis |

### Padrão de código HITL:

```python
result = agent.invoke(...)

while "__interrupt__" in result:
    # Mostrar ao usuário o que está sendo pedido
    # Coletar decisão (UI, input, automático, etc.)
    
    result = agent.invoke(
        Command(resume={"decisions": [{"type": "approve"}]}),
        config=config
    )
```

---

## 🎉 Parabéns! Você Completou o Curso LangChain Essentials!

### Resumo do que você aprendeu:

| Lab | Conceito | Descrição |
|-----|----------|-----------|
| **L1** | Fast Agent | Agentes ReAct com SQL |
| **L2** | Messages | Sistema de mensagens |
| **L3** | Streaming | Respostas em tempo real |
| **L4** | Tools | Criar ferramentas customizadas |
| **L5** | MCP | Conectar a servidores externos |
| **L6** | Memory | Memória de curto prazo |
| **L7** | Structured Output | Respostas em formato definido |
| **L8** | Dynamic Prompts | Adaptar comportamento |
| **L9** | HITL | Aprovação humana |

### Próximos passos:
- Explore a documentação do LangChain
- Construa seu próprio agente!
- Integre com MLflow para monitoramento
- Experimente com diferentes modelos no Databricks